In [129]:
import pandas as pd
from string import punctuation
import re
import spacy
from collections import defaultdict
import inflect

inflect = inflect.engine()

In [231]:
base_normal_path = 'LAS-NL-Explanations/sim_experiments/models/general/nli/test.tsv'
df = pd.read_csv(base_normal_path, delimiter = '\t')
df['t5-multi-exp-seed21'] = df.apply(lambda x: x[f't5-multi-exp-{x["preds_NLI_t5-base_ST.RA_seed21"]}-seed21'], axis=1)
df.columns

Index(['unique_key', 'label', 'premise', 'hypothesis', 'explanation1',
       'explanation2', 'explanation3', 'preds_NLI_t5-base_MT.RE_seed21',
       'preds_NLI_t5-base_MT.RA_seed21', 't5-single-exp-seed21',
       't5-MT-single-exp-seed21', 't5-MT-multi-exp-pred-seed21',
       'preds_NLI_t5-base_ST.RE_seed21',
       'preds_NLI_t5-base_sim.human_seed21_XE',
       'label_probs_NLI_t5-base_sim.human_seed21_XE',
       'preds_NLI_t5-base_sim.human_seed21_X',
       'label_probs_NLI_t5-base_sim.human_seed21_X',
       'preds_NLI_t5-base_sim.human_seed21_E',
       'label_probs_NLI_t5-base_sim.human_seed21_E',
       'preds_NLI_t5-base_sim.MT.RE_seed21_XE',
       'label_probs_NLI_t5-base_sim.MT.RE_seed21_XE',
       'preds_NLI_t5-base_sim.MT.RE_seed21_X',
       'label_probs_NLI_t5-base_sim.MT.RE_seed21_X',
       'preds_NLI_t5-base_sim.ST.RE_seed21_XE',
       'label_probs_NLI_t5-base_sim.ST.RE_seed21_XE',
       'preds_NLI_t5-base_sim.MT.RE_seed21_E',
       'label_probs_NLI_t5-base_

In [246]:
custom_punct = [c for c in punctuation]
custom_punct.remove("'")
print(custom_punct)
def clean(x):
    x = x.lower().strip()
    x = ''.join([c for c in x if c not in custom_punct])
    x = x.strip()
    x = x.replace('  ', ' ')
    return x
# preprocess
df['premise_clean'] = df.apply(lambda x: clean(x['premise']), axis=1)
df['hypo_clean'] = df.apply(lambda x: clean(x['hypothesis']), axis=1)
df['t5-MT-multi-exp-pred-seed21_clean'] = df.apply(lambda x: clean(x['t5-MT-multi-exp-pred-seed21']), axis=1)
df['t5-MT-single-exp-seed21_clean'] = df.apply(lambda x: clean(x['t5-MT-single-exp-seed21']), axis=1)
df['t5-single-exp-seed21_clean'] = df.apply(lambda x: clean(x['t5-single-exp-seed21']), axis=1)
df['t5-multi-exp-seed21_clean'] = df.apply(lambda x: clean(x['t5-multi-exp-seed21']), axis=1)
df.sample()

['!', '"', '#', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


,unique_key,label,premise,hypothesis,explanation1,explanation2,explanation3,preds_NLI_t5-base_MT.RE_seed21,preds_NLI_t5-base_MT.RA_seed21,t5-single-exp-seed21,...,t5-multi-exp-1-seed21,t5-multi-exp-2-seed21,preds_NLI_t5-base_ST.RA_seed21,t5-multi-exp-seed21,premise_clean,hypo_clean,t5-MT-multi-exp-pred-seed21_clean,t5-MT-single-exp-seed21_clean,t5-single-exp-seed21_clean,t5-multi-exp-seed21_clean
2131,6337406386.jpg#2r1c,2,A girl wearing khakis and a red sweatshirt is ...,"The girl is jumping in the air , while she is ...",The girl can be either in the forest or at the...,The girl can not be jumping in both a forest a...,There is n't a forest at the beach .,2,2,The girl can not be in the forest and at the b...,...,The forest is at the beach.,The girl can not be in the forest and at the b...,2,The girl can not be in the forest and at the b...,a girl wearing khakis and a red sweatshirt is ...,the girl is jumping in the air while she is at...,the girl can not be in the forest and at the b...,the girl can not be in the forest and at the b...,the girl can not be in the forest and at the b...,the girl can not be in the forest and at the b...


# General templates
"\<premise\>" 

"\<hypothesis\>" 

"\<hypothesis\> \<premise\>" 

"\<premise\> \<hypothesis\>" 

"Sentence 1 states \<premise\>. Sentence 2 is stating \<hypothesis\>" 

"Sentence 2 states <hypothesis>. Sentence 1 is stating \<premise\>" 
    
"There is \<hypothesis\>" 
    
"There is \<premise\>" 

In [248]:
explanations = ['t5-MT-single-exp-seed21_clean', 't5-MT-multi-exp-pred-seed21_clean', 't5-single-exp-seed21_clean', 't5-multi-exp-seed21_clean']
predictions = ['preds_NLI_t5-base_MT.RE_seed21', 'preds_NLI_t5-base_MT.RA_seed21', 'preds_NLI_t5-base_ST.RE_seed21', 'preds_NLI_t5-base_ST.RA_seed21']

for expl in explanations:
    counter = 0
    for i, row in df.iterrows():
        if row[expl] == row['hypo_clean']:
            
            counter += 1
    print(expl, counter)

t5-MT-single-exp-seed21_clean 1
t5-MT-multi-exp-pred-seed21_clean 9
t5-single-exp-seed21_clean 23
t5-multi-exp-seed21_clean 12


In [132]:
explanations = ['t5-MT-single-exp-seed21_clean', 't5-MT-multi-exp-pred-seed21_clean', 't5-single-exp-seed21_clean', 't5-multi-exp-seed21_clean']
predictions = ['preds_NLI_t5-base_MT.RE_seed21', 'preds_NLI_t5-base_MT.RA_seed21', 'preds_NLI_t5-base_ST.RE_seed21', 'preds_NLI_t5-base_ST.RA_seed21']

for expl in explanations:
    counter = 0
    for i, row in df.iterrows():
        if row[expl] == row['premise_clean'] or \
            row[expl] == row['hypo_clean'] or \
            row[expl] == row['hypo_clean'] + ' ' + row['premise_clean'] or \
            row[expl] == row['premise_clean'] + ' ' + row['hypo_clean'] or \
            row[expl] == 'there is ' + row['hypo_clean'] or \
            row[expl] == 'there is ' + row['premise_clean'] or \
            row[expl] == f"sentence 1 states {row['premise_clean']}. sentence 2 is stating {row['hypo_clean']}." or \
            row[expl] == f"sentence 1 states {row['hypo_clean']}. sentence 2 is stating {row['premise_clean']}.":
#             print(row[expl])
            counter += 1
    print(expl, counter)

t5-MT-single-exp-seed21_clean 21
t5-MT-multi-exp-pred-seed21_clean 57
t5-single-exp-seed21_clean 89
t5-multi-exp-seed21_clean 64


# Templates

In [233]:
from collections import OrderedDict
tuple_regex = OrderedDict()
tuple_regex['contradiction', 'different than'] = r"^(?P<s1>.*)\ (is|are)\ different than\ (?P<s2>.*)"
tuple_regex['contradiction', 'is different'] = r'^(?P<s1>.*)\ and\ (?P<s2>.*) (is|are)\ different'
tuple_regex['contradiction', 'would not be able'] = r'^(?P<s1>.*)\ would not be able to\ (?P<s2>.*)'
tuple_regex['contradiction', 'is not the same that'] = r"^(?P<s1>.*)\ (does not|doesnt|doesn't|dont|don't|do not)\ (imply that|imply)\ (?P<s2>.*)"
tuple_regex['contradiction', 'is not the same'] = r"^(?P<s1>.*)\ (is|are)\ (not the same as|not|the opposite of)\ (?P<s2>.*)"
tuple_regex['contradiction', 'cant at the same time'] = r"(cant|can't|cannot|can not)\ be\ (?P<s1>.*)\ and\ (?P<s2>.*)\ (simultaneously|at the same time|at once)"
tuple_regex['contradiction', 'cannot be'] = r"^(?P<s1>.*)\ (cant|can't|cannot|can not)\ be\ (?P<s2>.*)"
tuple_regex['contradiction', 'is not a'] = r"^(?P<s1>.*)\ (is|are)\ not\ (a|an)\ (?P<s2>.*)"
tuple_regex['contradiction', 'is not'] = r"^(?P<s1>.*)\ (is|are)\ not\ (?P<s2>.*)"
tuple_regex['contradiction', 'cant be if'] = r"^(cant|can't|cannot|can not)\ be\ (?P<s1>.*)\ if\ (is|are)\ (?P<s2>.*)"
tuple_regex['contradiction', 'cant if'] = r"^(cant|can't|cannot|can not)\ (?P<s1>.*)\ if\ (?P<s2>.*)"
#     ('contradiction', 'cant be if he'): r"(cant|can't|cannot|can not)\ be\ (?P<s1>.*)\ if\ (he|she|they)\ (is|are)\ (?P<s2>.*)"
tuple_regex['contradiction', 'does not mean'] = r"^(just because)?\ (?P<s1>.*)\ (does not|doesn't|doesnt)\ mean that\ (?P<s2>.*)"
tuple_regex['contradiction', 'does not mean that'] = r"^(just because)?\ (?P<s1>.*)\ (does not|doesn't|doesnt)\ mean\ (?P<s2>.*)"
tuple_regex['neutral', 'no way to know A or B'] = r"^no way to know\ (?P<s1>.*)\ or\ (?P<s2>.*)"
tuple_regex['neutral', 'there is more A than B'] = r"^there is more\ (?P<s1>.*)\ than\ (?P<s2>.*)"
tuple_regex['neutral', 'can A without B'] = r"^can\ (?P<s1>.*)\ without\ (?P<s2>.*)"
tuple_regex['neutral', 'could be A not just B'] = r"^could be\ (?P<s1>.*)\ not just\ (?P<s2>.*)"
tuple_regex['neutral', 'there is not evidence A or B'] = r"^there is no evidence\ (?P<s1>.*)\ or\ (?P<s2>.*)"
tuple_regex['neutral', 'A might not be B'] = r"^(?P<s1>.*)\ might not be\ (?P<s2>.*)"
tuple_regex['neutral', 'not everyone A will B'] = r"^not\ (everyone|every)\ (?P<s1>.*)\ will\ (?P<s2>.*)"
tuple_regex['neutral', 'we can’t tell if A is B'] = r"^we\ (can't|cannot|can not)\ tell if\ (?P<s1>.*)\ (is|are)\ (?P<s2>.*)"
tuple_regex['neutral', 'not all A are B'] = r"^(not all|not every)\ (?P<s1>.*)\ (is|are)\ (?P<s2>.*)"
tuple_regex['neutral', 'just because A does not mean B'] = r"^just because\ (?P<s1>.*)\ (does not|doesnt|doesn't)\ (necessarily mean|mean|necessarily make|make|necessarily imply|imply|necessarily indicate|indicate)\ (?P<s2>.*)"
tuple_regex['neutral', 'A is not necessarily B'] = r"^(?P<s1>.*)\ (is|are)\ not necessarily\ (?P<s2>.*)"
tuple_regex['neutral', 'A doe not have to be B'] = r"^(?P<s1>.*)\ (does not|doesnt|doesn't|don't|dont|do not)\ have to be\ (?P<s2>.*)"
tuple_regex['neutral', 'A does not necessarily B'] = r"^(?P<s1>.*)\ (does not|doesnt|doesn't|don't|dont|do not)\ necessarily\ (?P<s2>.*)"
tuple_regex['neutral', 'does not necessarily indicate that'] = r"^(?P<s1>.*)\ (does not|doesnt|doesn't|don't|dont|do not)\ (necessarily mean|mean|necessarily make|make|imply|necessarily indicate|indicate) that\ (?P<s2>.*)"
tuple_regex['neutral', 'just does not necessarily indicate'] = r"^just because\ (?P<s1>.*)\ (does not|doesnt|doesn't|don't|dont|do not)\ (necessarily mean|mean|necessarily make|make|necessarily imply|imply|necessarily indicate|indicate)\ (?P<s2>.*)"
tuple_regex['neutral', 'does not necessarily indicate'] = r"(?P<s1>.*)\ (does not|doesnt|doesn't|don't|dont|do not)\ (necessarily mean|mean|necessarily make|make|necessarily imply|imply|necessarily indicate|indicate)\ (?P<s2>.*)"
tuple_regex['neutral', 'we dont know A to B'] = r"^we\ (did not|didn't|didnt|dont|do not|don't)\ know\ (?P<s1>.*)\ to\ (?P<s2>.*)"
tuple_regex['neutral', 'this does not imply A or B'] = r"^this\ (does not|doesnt|doesn't) imply\ (?P<s1>.*)\ or\ (?P<s2>.*)"
tuple_regex['neutral', 'A and B R1 two different'] = r"^(?P<s1>.*)\ and\ (?P<s2>.*)\ (is|are)\ (two different|different)"
tuple_regex['neutral', 'not everyone A will B'] = r"^not everyone\ (?P<s1>.*)\ (will|would)\ (?P<s2>.*)"
tuple_regex['neutral', 'it cannot be assumed that A is B'] = r"^it cannot be assumed that\ (?P<s1>.*)\ (is|are)\ (?P<s2>.*)"
tuple_regex['neutral', 'some A or B'] = r"^some\ (?P<s1>.*)\ or\ (?P<s2>.*)"
tuple_regex['neutral', 'does not have to be A to B'] = r"^(does not|doesnt|doesn't|dont|don't|do not)\ have to be\ (?P<s1>.*)\ (or|to)\ (?P<s2>.*)"
tuple_regex['entailment', 'implies that'] = r'^(?P<s1>.*)\ (implies that|implies|means|would imply)\ (?P<s2>.*)'
tuple_regex['entailment', 'if,then'] = r'^if (?P<s1>.*)\ then\ (?P<s2>.*)'
tuple_regex['entailment', 'same as'] = r'^(?P<s1>.*)\ (is|are)\ (the same as|same as|a rephrasing of)\ (?P<s2>.*)'
tuple_regex['entailment', 'would be'] = r'^(?P<s1>.*)\ would be\ (?P<s2>.*)'
tuple_regex['entailment', 'can also be said as'] = r'^(?P<s1>.*)\ (can|could) also be said as\ (?P<s2>.*)'
tuple_regex['entailment', 'is the same thing'] = r'^(?P<s1>.*)\ and\ (?P<s2>.*)\ (is|are)\ the same thing'
tuple_regex['entailment', 'so'] = r'^(?P<s1>.*)\ (so|then)\ (?P<s2>.*)'
tuple_regex['entailment', 'synonyms'] = r'^(?P<s1>.*)\ and\ (?P<s2>.*)\ are synonyms'
tuple_regex['entailment', 'isatype'] = r'^(?P<s1>.*)\ (is|are)\ a type of\ (?P<s2>.*)'
tuple_regex['entailment', 'isaway'] = r'^(?P<s1>.*)\ is a way of saying\ (?P<s2>.*)'
tuple_regex['entailment', 'is synonymous'] = r'^(?P<s1>.*)\ (is|are)\ synonymous with\ (?P<s2>.*)'
tuple_regex['entailment', 'is a form of'] = r'^(?P<s1>.*)\ (is|are)\ (a|another)\ form of\ (?P<s2>.*)'
tuple_regex['entailment', 'isan'] = r'^(?P<s1>.*)\ (is an|is a|are)\ (?P<s2>.*)'
tuple_regex['entailment', 'is/are'] = r'^(?P<s1>.*)\ (is|are|have to be|has to be|must be)\ (?P<s2>.*)'


triple_regex = {
    #NO Subject/VERB a man can not be reading the paper and climbing a mountain at the same time
    ('contradiction', 'at the same time'): r'^(?P<s1>.*)\ can not be\ (?P<s2>.*)\ and\ (?P<s3>.*)\ (at the same time|simultaneously)'
    
}

In [234]:
def subject_verb(text):
    doc=nlp(text)
    sub_toks = [tok for tok in doc if (tok.dep_ in ["nsubj",'nsubjpass', 'expl']) or (tok.dep_ == 'ROOT' and tok.pos_=='NOUN')] #(woman, 'NOUN', 'ROOT')
    verbs = [tok for tok in doc if tok.pos_ in ['VERB','AUX']]
    if len(sub_toks) > 0 and len(verbs) > 0:
        return True
    
    return False

In [235]:
for expl, model_pred in zip(explanations, predictions):
    count = defaultdict(lambda: 0)
    with open(f'snli_{expl}_patterns.tsv', 'w') as out:
        out.write('unique_key\tpremise\thypothesis\tpred_explanation\tpattern\tpred_label\tlabel\n')
        for i, row in df.iterrows():
            found = False
            premise_tok = set([w.lower() for w in row['premise'].split(' ') if w not in punctuation])
            hypo_tok = set([w.lower() for w in row['hypothesis'].split(' ') if w not in punctuation])
            
            for regex_name, regex in tuple_regex.items():
                matches = re.search(regex, row[expl])
                if matches:
                    if subject_verb(matches.group('s1')) and subject_verb(matches.group('s2')):
                        g1_tok = set([w.lower() for w in matches.group('s1').split(' ') if w not in punctuation])
                        g2_tok = set([w.lower() for w in matches.group('s2').split(' ') if w not in punctuation])
                        if len(g1_tok.intersection(premise_tok)) > len(g2_tok.intersection(premise_tok)):
                            new_premise = matches.group('s1')
                            new_hypothesis = matches.group('s2')
                        else:
                            new_premise = matches.group('s2')
                            new_hypothesis = matches.group('s1')
                        out.write(f"{row['unique_key']}\t{new_premise.capitalize()}.\t{new_hypothesis.capitalize()}.\t{row[expl.strip('_clean')]}\t{regex_name[1]}\t{row[model_pred]}\t{row['label']}\n")
                        count[regex_name[0]] += 1
                        found = True
                        break
        print(expl, count)

t5-MT-single-exp-seed21_clean defaultdict(<function <lambda> at 0x137d92b80>, {'entailment': 876, 'contradiction': 1497, 'neutral': 242})
t5-MT-multi-exp-pred-seed21_clean defaultdict(<function <lambda> at 0x28e4bcaf0>, {'contradiction': 1674, 'entailment': 886, 'neutral': 325})
t5-single-exp-seed21_clean defaultdict(<function <lambda> at 0x16ddedee0>, {'contradiction': 1320, 'entailment': 758, 'neutral': 333})
t5-multi-exp-seed21_clean defaultdict(<function <lambda> at 0x28e554ee0>, {'entailment': 737, 'contradiction': 1835, 'neutral': 124})


In [236]:
for expl, model_pred in zip(explanations, predictions):
    count = 0
    with open(f'snli_{expl}_patterns.tsv', 'a') as out:
        for i, row in df.iterrows():
            found = False
            for regex_name, regex in triple_regex.items():
                matches = re.search(regex, row[expl])
                if matches:
                    
                    doc=nlp(matches.group('s1'))
                    plurals = [inflect.singular_noun(tok.text) for tok in doc]
                    aux_verb = 'is'
                    if any(w != False for w in plurals):
                        aux_verb = 'are'
                    s1 = matches.group('s1') + f' {aux_verb} ' + matches.group('s2')
                    s2 = matches.group('s1') + f' {aux_verb} ' + matches.group('s3')
                    if subject_verb(s1) and subject_verb(s2):
                        out.write(f"{row['unique_key']}\t{s1}\t{s2}\t{row[expl].strip('_clean')}\tcontr_triple\t{row[model_pred]}\t{row['label']}\n")
                        
                        count += 1
                        found = True
                        break
    print(expl, count)
    

t5-MT-single-exp-seed21_clean 1332
t5-MT-multi-exp-pred-seed21_clean 1604
t5-single-exp-seed21_clean 1590
t5-multi-exp-seed21_clean 1637


# Compare predictions and explanations on the original and the reconstructed input

In [ ]:
base_normal_path = 'LAS-NL-Explanations/sim_experiments/models/general/nli/test.tsv'
df = pd.read_csv(base_normal_path, delimiter = '\t')
df['t5-multi-exp-seed21'] = df.apply(lambda x: x[f't5-multi-exp-{x["preds_NLI_t5-base_ST.RA_seed21"]}-seed21'], axis=1)
df.columns

base_normal_path = 'LAS-NL-Explanations/sim_experiments/models/general/nli/test_patterns.tsv'
df_p = pd.read_csv(base_normal_path, delimiter = '\t')
df_p.sample()

In [100]:
df_p['orig'] = df.apply(lambda x: df[df['unique_key']==row['unique_key']]['preds_NLI_t5-base_MT.RE_seed21'].values[0], axis=1) 
df_p.sample()

,unique_key,label,premise,hypothesis,explanation1,explanation2,explanation3,preds_NLI_t5-base_MT.RE_seed21,preds_NLI_t5-base_MT.RA_seed21,t5-single-exp-seed21,...,label_probs_NLI_t5-base_sim.MT.RA_seed21_XE,preds_NLI_t5-base_sim.MT.RA_seed21_X,label_probs_NLI_t5-base_sim.MT.RA_seed21_X,preds_NLI_t5-base_sim.MT.RA_seed21_E,label_probs_NLI_t5-base_sim.MT.RA_seed21_E,t5-multi-exp-0-seed21,t5-multi-exp-1-seed21,t5-multi-exp-2-seed21,preds_NLI_t5-base_ST.RA_seed21,orig
4933,3411579899.jpg#4r1c,0,The cyclist rides on a wooded path .,The cyclist rides with a companion on a wooded...,A cyclist can ride without a companion .,Riding on a wooded path does not imply the cyc...,Just because the cyclist rides on a wooded pat...,NaN,0,Not all cyclists ride with a companion.,...,0.355966,0,0.339347,0,0.351766,Not all cyclists ride with a companion.,The cyclist rides on a wooded path is the same...,The cyclist can not be riding with a companion.,0,0


In [102]:
(df_p['orig'] == df['preds_NLI_t5-base_MT.RE_seed21']).value_counts(normalize=True)

False    0.663274
True     0.336726
dtype: float64

In [241]:
base_normal_path = 'LAS-NL-Explanations/sim_experiments/models/general/nli/snli_t5-MT-single-exp-seed21_clean_patterns.tsv'
df_new = pd.read_csv(base_normal_path, delimiter = '\t')
pred = 'preds_NLI_t5-base_MT.RE_seed21'
print((df_new['pred_label'] == df_new[pred]).value_counts(normalize=True))
print((df_new['pred_label'] == df_new[pred]).value_counts())
df_change = df_new[df_new['pred_label'] == df_new[pred]]
print((df_change['label']==df_change['pred_label']).value_counts(normalize=True))
print((df_change['label']==df_change[pred]).value_counts(normalize=True))

True     0.923233
False    0.076767
dtype: float64
True     3644
False     303
dtype: int64
True     0.904226
False    0.095774
dtype: float64
True     0.904226
False    0.095774
dtype: float64


In [242]:
base_normal_path = 'LAS-NL-Explanations/sim_experiments/models/general/nli/snli_t5-single-exp-seed21_clean_patterns.tsv'
df_new = pd.read_csv(base_normal_path, delimiter = '\t')
pred = 'preds_NLI_t5-base_ST.RE_seed21'
print((df_new['pred_label'] == df_new[pred]).value_counts(normalize=True))
print((df_new['pred_label'] == df_new[pred]).value_counts())
df_change = df_new[df_new['pred_label'] == df_new[pred]]
print((df_change['label']==df_change['pred_label']).value_counts(normalize=True))
print((df_change['label']==df_change[pred]).value_counts(normalize=True))

True     0.903274
False    0.096726
dtype: float64
True     3614
False     387
dtype: int64
True     0.900111
False    0.099889
dtype: float64
True     0.900111
False    0.099889
dtype: float64


In [243]:
base_normal_path = 'LAS-NL-Explanations/sim_experiments/models/general/nli/snli_t5-MT-multi-exp-pred-seed21_clean_patterns.tsv'
df_new = pd.read_csv(base_normal_path, delimiter = '\t')
pred = 'preds_NLI_t5-base_MT.RA_seed21'
print((df_new['pred_label'] == df_new[pred]).value_counts(normalize=True))
print((df_new['pred_label'] == df_new[pred]).value_counts())
df_change = df_new[df_new['pred_label'] == df_new[pred]]
print((df_change['label']==df_change['pred_label']).value_counts(normalize=True))
print((df_change['label']==df_change[pred]).value_counts(normalize=True))

True     0.922254
False    0.077746
dtype: float64
True     4140
False     349
dtype: int64
True     0.891304
False    0.108696
dtype: float64
True     0.891304
False    0.108696
dtype: float64


In [ ]:
base_normal_path = 'LAS-NL-Explanations/sim_experiments/models/general/nli/snli_t5-multi-exp-seed21_clean_patterns.tsv'
df_new = pd.read_csv(base_normal_path, delimiter = '\t')
pred = 'preds_NLI_t5-base_ST.RA_seed21'
print((df_new['pred_label'] == df_new[pred]).value_counts(normalize=True))
print((df_new['pred_label'] == df_new[pred]).value_counts())
df_change = df_new[df_new['pred_label'] == df_new[pred]]
print((df_change['label']==df_change['pred_label']).value_counts(normalize=True))
print((df_change['label']==df_change[pred]).value_counts(normalize=True))